In [1]:
import pathlib
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

#Data 
import math
import numpy as np
import datetime as dt
import pandas as pd
import json

In [2]:
dfSE = pd.read_csv(r'C:\Users\nanag\PycharmProjects\A4A\Code\Data\pac\SE.csv', sep=';',encoding='latin1')
dfPAC = pd.read_csv(r'C:\Users\nanag\PycharmProjects\A4A\Code\Data\pac\pac.txt', sep='\t',encoding='latin1')
varFilterMunicipio = ''


In [3]:
if len(varFilterMunicipio) > 1:
    dfSE_1 = dfSE[dfSE['Municipio'].isin(varFilterMunicipio)]
    dfPAC_1 = dfPAC[dfPAC['DIV_CNOM_MUNICIPIO'].isin(varFilterMunicipio)]
else:
    dfSE_1 = dfSE
    dfPAC_1 = dfPAC


In [9]:
dfSE_municipio= dfSE_1
dfSE['Número de usuarios'] = dfSE['numeroDocumento']
dfSE_municipio = dfSE_municipio.drop_duplicates(['Número de usuarios'])
dfSE_municipio['Año/mes terminación laboral'] = [("_" + i[0:4] + i[5:7]) for i in dfSE['Fecha de terminación laboral']]
dfSE_municipio['Año/mes terminación laboralInt'] = [int(i[1:]) for i in dfSE_municipio['Año/mes terminación laboral']]
dfSE_municipio = dfSE_municipio[dfSE_municipio["Año/mes terminación laboralInt"] >= 201802]
SE_Municipio = dfSE_municipio.groupby(['Año/mes terminación laboral', 'Municipio']).count().reset_index()
Line_fig = px.bar(SE_Municipio, x="Año/mes terminación laboral", y="Número de usuarios", color="Municipio")
Line_fig.update_layout(title='SE Municipio', paper_bgcolor="#F8F9F9").update_xaxes(categoryorder = "category ascending")

In [10]:
dfPAC_municipio = dfPAC_1
dfPAC_1['Número de usuarios'] = dfPAC_1['MPC_NID']
dfPAC_1['Municipios'] = dfPAC_1['DIV_CNOM_MUNICIPIO']
dfPAC_municipio['Año/mes radicación solicitud beneficiario'] = (dfPAC_1['FEC_APROBACION_BENEFICIARIO'] / 100)
dfPAC_municipio['Año/mes radicación solicitud beneficiario'] = [int(i) for i in dfPAC_1['Año/mes radicación solicitud beneficiario']]
dfPAC_municipio = dfPAC_1.groupby(['Año/mes radicación solicitud beneficiario', 'Municipios']).count().reset_index()
dfPAC_municipio = dfPAC_municipio[dfPAC_municipio['Año/mes radicación solicitud beneficiario'] > 201712]
dfPAC_municipio['Año/mes radicación solicitud beneficiario'] = ["_" + str(i) for i in dfPAC_municipio['Año/mes radicación solicitud beneficiario']]
Scatter_fig = px.bar(dfPAC_municipio, x="Año/mes radicación solicitud beneficiario", y="Número de usuarios", color="Municipios")
Scatter_fig.update_layout(title='PaC Municipio', paper_bgcolor="#F8F9F9")


In [7]:
SE_municipio_e = dfSE_municipio[['Municipio', 'Año/mes terminación laboral']] 
#SE_municipio_e = SE_municipio_e[SE_municipio_e['Zona'] == 'Rural']
SE_municipio_e = SE_municipio_e.groupby(['Municipio']).count().reset_index() 
SE_municipio_e.head()

,Municipio,Año/mes terminación laboral
0,ABEJORRAL,6
1,ALEJANDRIA,3
2,AMAGA,4
3,AMALFI,2
4,ANDES,4


In [8]:
PAC_municipio_e = dfPAC_municipio[['DIV_CNOM_MUNICIPIO', 'Año/mes radicación solicitud beneficiario']] 
PAC_municipio_e = PAC_municipio_e.groupby(['DIV_CNOM_MUNICIPIO']).count().reset_index() 
PAC_municipio_e.head()

,DIV_CNOM_MUNICIPIO,Año/mes radicación solicitud beneficiario
0,ABEJORRAL,2
1,ABREGO,3
2,ACACiAS,18
3,ACANDÍ,1
4,ACEVEDO,2


In [8]:
aux = SE_municipio_e.merge(PAC_municipio_e, left_on='Municipio', right_on='DIV_CNOM_MUNICIPIO')

aux.head()

,Municipio,Año/mes terminación laboral,DIV_CNOM_MUNICIPIO,Año/mes radicación solicitud beneficiario
0,ABEJORRAL,6,ABEJORRAL,2
1,AMALFI,2,AMALFI,4
2,ANDES,4,ANDES,8
3,ANGELOPOLIS,3,ANGELOPOLIS,1
4,ANGOSTURA,2,ANGOSTURA,3


In [9]:
import scipy.stats
x = np.array(aux['Año/mes terminación laboral'])
y = np.array(aux['Año/mes radicación solicitud beneficiario'])
x

array([  6,   2,   4,   3,   2,   1,   5,   1,  16, 357,   4,   5,   3,
         1,  66,   5,  11,   4,   2,  46,   2,   1, 116,   3,   4,  21,
         1,  24,   4,   1, 232,  52,  57,   8,   1,  25,   7,   1,  16,
         3,   2,  12, 138,  52,   1,   2,   2,   7,   2,   2,   3,   1,
         1,   6,   6,   2,   7,   2], dtype=int64)

In [10]:
y

array([ 2,  4,  8,  1,  3,  3,  1, 19, 11, 23,  2,  3,  3,  1, 13, 11, 12,
        8,  6, 11,  7,  1, 14,  1,  4, 10, 13,  9,  2,  1, 20, 14, 10,  4,
        2, 10,  1,  1,  5,  5,  2,  6, 19, 13,  1,  9,  1,  6,  3,  3,  2,
        4, 25,  2, 14,  2, 10,  4], dtype=int64)

In [11]:
scipy.stats.pearsonr(x, y)

(0.6133652396577631, 3.072349632779293e-07)

In [12]:
print(scipy.stats.spearmanr(x, y))

SpearmanrResult(correlation=0.5286042281987791, pvalue=1.9964480661882218e-05)


In [13]:
print(scipy.stats.kendalltau(x, y))

KendalltauResult(correlation=0.4381056569029996, pvalue=5.659558820554022e-06)
